In [1]:
%%writefile superBroadcast.py

from mpi4py import MPI
import numpy as np
import json

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
rankNum = comm.Get_size()

print("\n"+str(rank)+"/"+str(rankNum))

if rank==0:        # example [0,[0,((0,0,1),1)],[1,((0,1,0),0)],[2]] with rankNum => 3
    mToBcast=[0,[0,((0,0,1),1)],
            [1,((0,1,0),0)]]
    for k in range(2,rankNum):
        mToBcast.append([k])
    #print(mBcast)

if rank!=0:        # example [1|2,[0],[1],[2]] with rankNum -> 3
    mToBcast=[rank]
    for k in range(rankNum):
        mToBcast.append([k])
    #print(mBcast)
    
mToBcast=json.dumps(mToBcast)
mToBcast=np.array(mToBcast, dtype='S50')
mToBcast=mToBcast.tobytes()
print( "before bcast", rank, mToBcast)

data=[""]*rankNum
for k in range(rankNum):
    if rank == k:
        data[k] = mToBcast
    else:
        data[k] = bytearray(50)
for k in range(rankNum):
    comm.Bcast(data[k], root=k)

for k in range(rankNum):
    data[k]=data[k].decode().rstrip('\x00')

for k in range(rankNum):
    data[k]=json.loads(data[k])

print("\nafter bcast", rank, data)

    


Overwriting superBroadcast.py


In [2]:
! mpirun -n 3 python3 superBroadcast.py


0/3
before bcast 0 b'[0, [0, [[0, 0, 1], 1]], [1, [[0, 1, 0], 0]], [2]]'

after bcastIII 0 [[0, [0, [[0, 0, 1], 1]], [1, [[0, 1, 0], 0]], [2]], [1, [0], [1], [2]], [2, [0], [1], [2]]]

1/3
before bcast 1 b'[1, [0], [1], [2]]\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

after bcastIII 1 [[0, [0, [[0, 0, 1], 1]], [1, [[0, 1, 0], 0]], [2]], [1, [0], [1], [2]], [2, [0], [1], [2]]]

2/3
before bcast 2 b'[2, [0], [1], [2]]\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

after bcastIII 2 [[0, [0, [[0, 0, 1], 1]], [1, [[0, 1, 0], 0]], [2]], [1, [0], [1], [2]], [2, [0], [1], [2]]]
